# 🚦 Validación y Exploración Inicial de Datos de Tráfico

Este notebook tiene como objetivo:
1.  Conectar con la carpeta de datos (`data/raw`).
2.  Verificar que se están descargando archivos JSON.
3.  Inspeccionar la estructura de los datos (schemas, claves).
4.  Validar la calidad básica (¿tenemos datos de tráfico reales?).

In [ ]:
import os
import json
import pandas as pd
from datetime import datetime
from pathlib import Path

# CONFIGURACIÓN DE RUTAS
# ----------------------
# Como el notebook corre en el VPS dentro de un contenedor o directamente sobre /opt,
# debemos apuntar a donde Jupyter "ve" los datos.
# Asumimos que Jupyter se inicia en la raíz del proyecto.
DATA_PATH = Path("../data/raw")

print(f"📂 Buscando datos en: {DATA_PATH.resolve()}")

In [ ]:
def listar_archivos(base_path):
    files_found = []
    if not base_path.exists():
        print(f"❌ La ruta {base_path} no existe.")
        return []
        
    for file in base_path.rglob("*.json"):
        files_found.append(file)
    # Ordenar por nombre garantiza orden cronológico si la estructura es YYYY/MM/DD/HHMM.json
    return sorted(files_found)

files = listar_archivos(DATA_PATH)
print(f"📂 Archivos encontrados: {len(files)}")

if files:
    # Mostrar los últimos 3 para confirmar fechas
    print("\n📅 Últimos 3 archivos:")
    for f in files[-3:]:
        print(f" - {f.name} (Modificado: {datetime.fromtimestamp(f.stat().st_mtime)})")

In [ ]:
if files:
    # Leemos el último archivo para ver la estructura
    latest_file = files[-1]
    print(f"\n📄 Inspeccionando EL ÚLTIMO ARCHIVO: {latest_file}")
    
    with open(latest_file, 'r') as f:
        data = json.load(f)
    
    # CARGA CORRECTA usando 'results' (Estructura Opendatasoft v2.1)
    # json_normalize "aplana" el geo_shape para que sea legible en tabla
    if 'results' in data:
        df = pd.json_normalize(data['results'])
        print(f"📊 Registros encontrados: {df.shape[0]} (Esperado ~400)")
        print(f"📊 Columnas: {df.shape[1]}")
        display(df.head())
    else:
        print(f"⚠️ No se encontró la clave 'results'. Claves disponibles: {list(data.keys())}")

In [ ]:
if 'df' in locals():
    # 🕵️‍♂️ INSPECCIÓN DE CALIDAD
    print("\n🧐 ¿Tenemos datos de tráfico?")
    print("-" * 30)
    
    # Verificar si la columna 'estado' tiene algún valor que no sea null
    # Ajuste: A veces la columna se llama 'estado', 'status', o 'jam_factor'. 
    # Imprimimos columnas para asegurar.
    # print("Columnas detectadas:", df.columns.tolist())
    
    target_col = 'estado' # Ajustar según output anterior si difiere
    
    if target_col in df.columns:
        con_datos = df[target_col].notnull().sum()
        print(f"Filas con '{target_col}' informado: {con_datos} de {len(df)}")
        if con_datos > 0:
            print("Ejemplos de estados:", df[target_col].unique())
    else:
        print(f"❌ ¡Alerta! No existe la columna '{target_col}'. Revisa la lista de columnas arriba.")

    # Ver tipos de datos
    print("\nℹ️ Info del DataFrame:")
    df.info()